In [1]:
# Import modules for our task

import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, normalize

In [11]:
# Import system libraries and our Scripts

import os
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from user_overview_script import UserOverviewScript
from df_cleaning import DataFrameCleaning
from df_info import DataFrameInfo
from df_outlier import *
from plots import *

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [35]:
# Read Engagment and Experience Telecom dataset

user_engagement  = pd.read_csv('../data/user_engagement.csv')
user_experiance = pd.read_csv('../data/user_experiance.csv')

In [20]:
user_engagement.head()

,Customer_Id,Cluster,Session_Frequency,Duration,Total_Data_Volume
0,33601001722.00,3,1.00,116720.00,878690574.00
1,33601001754.00,2,1.00,181230.00,156859643.00
2,33601002511.00,2,1.00,134969.00,595966483.00
3,33601007832.00,0,1.00,49878.00,422320698.00
4,33601008617.00,1,2.00,37104.00,1457410944.00


In [21]:
user_experiance.head()

,MSISDN_Number,Cluster,Total_Avg_RTT,Total_Avg_Bearer_TP,Total_Avg_TCP
0,33601001722.00,0,0.05,76.00,3919.15
1,33601001754.00,1,0.03,99.00,1119.76
2,33601002511.00,1,0.05,97.00,1119.76
3,33601007832.00,1,0.08,248.00,366.24
4,33601008617.00,2,0.12,56844.00,1858.68


### **Task 4** - User Satisifaction Analysis
 Based on the Engagement Analysis and the Experience Analysis we have conducted in previous tasks, we will analyze the User Satisifaction in this task.
**Task 4.1.** - Write a python program to assign:
* An engagement score to each user, and
* An experience score to each user

##### Assign User Engagement Score

In [36]:
# Read the saved user engagement model

with open("../models/user_engagement.pkl", "rb") as f:
    kmeans1 = pickle.load(f)
less_engaged_score = 3

In [37]:
engagement_df = user_engagement.set_index('Customer_Id')[
    ['Duration', 'Total_Data_Volume', 'Session_Frequency']]
scaler = StandardScaler()
scaled_array = scaler.fit_transform(engagement_df)
pd.DataFrame(scaled_array).head(5)

,0,1,2
0,-0.07,0.54,-0.55
1,0.74,-1.28,-0.55
2,0.16,-0.17,-0.55
3,-0.92,-0.61,-0.55
4,-1.08,2.00,0.98


In [38]:
data_normalized = normalize(scaled_array)
pd.DataFrame(data_normalized).head(5)

,0,1,2
0,-0.09,0.70,-0.71
1,0.47,-0.81,-0.35
2,0.26,-0.29,-0.92
3,-0.74,-0.50,-0.45
4,-0.44,0.81,0.39


In [39]:
distance = kmeans1.fit_transform(data_normalized)
distance_from_less_engaged_score = list(
    map(lambda x: x[less_engagement], distance))
user_engagement['Engagement_Score'] = distance_from_less_engaged_score
user_engagement.head(5)

,Customer_Id,Cluster,Session_Frequency,Duration,Total_Data_Volume,Engagement_Score
0,33601001722.00,3,1.00,116720.00,878690574.00,0.76
1,33601001754.00,2,1.00,181230.00,156859643.00,1.53
2,33601002511.00,2,1.00,134969.00,595966483.00,1.09
3,33601007832.00,0,1.00,49878.00,422320698.00,0.72
4,33601008617.00,1,2.00,37104.00,1457410944.00,1.19
